In [8]:
# Disminuye la resolución espacial de datos horarios.

import pandas as pd
import xarray as xr

fdir = [ "../data/", "../temp/",
    "../results/comp/prec_mroc_h20a_hist_prec_mroc_2040_2040/" ]
name = [ "prec_era5_hist_hist.nc", "prec_mroc_hist_hist.nc", 
    "prec_mroc_h20a_hist.nc", "prec_mroc_2040_2040.nc", 
    "prec_hist_hist.nc", "prec_mroc_h20a_hist_prec_mroc_2040_2040.nc" ]

k = 5
i = 3
n = 5

f = 0
lon,   lat   = "XLONG",     "XLAT"
lon_d, lat_d = "west_east", "south_north" 
if i == 5:
    f = 2
    lon,   lat   = "LONGITUD", "LATITUD"
    lon_d, lat_d = "LONGITUD", "LATITUD"

ds = xr.open_dataset(fdir[f] + name[i])
if k != 0: ds = ds.isel( { lon_d: slice(k, -k), lat_d: slice(k, -k) } )
if i != 5:
    ds[lon] = ds[lon].isel( {lat_d: 0} )
    ds[lat] = ds[lat].isel( {lon_d: 0})
    ds = ds.swap_dims( {lon_d: lon, lat_d: lat} )
ds = ds.coarsen( {lat: n, lon: n}, boundary = "trim" ).mean()
if i != 5: ds = ds.rename_dims( {lon: lon_d, lat: lat_d} )
ds.to_netcdf(fdir[1] + name[i][:-3] + "_coarsen_" + str(n) + ".nc")

In [7]:
# Hace compatible la resolución espacial de CHIRPS con WRF.

import pandas as pd
import xarray as xr

ds_1 = xr.open_dataset("../temp/prec_hist_hist_days.nc")
ds_1["XLONG"] = ds_1["XLONG"].isel(south_north = 0)
ds_1["XLAT"] = ds_1["XLAT"].isel(west_east = 0)

ds_1 = ds_1.swap_dims( {"west_east": "XLONG", "south_north": "XLAT"} )

ds_2 = xr.open_dataset( "../temp/CHIRPS_comp_hist.nc"
    ).rename_dims( {"time": "XTIME", "longitude": "XLONG",
    "latitude": "XLAT"} ).rename_vars( {"time": "XTIME",
    "longitude": "XLONG", "latitude": "XLAT", "precip": "Pcp"}
    ).interp( {"XLAT": ds_1["XLAT"], "XLONG": ds_1["XLONG"]},
    method = "cubic" ).rename_dims( {"XLAT": "latitude",
    "XLONG": "longitude", "XTIME": "time"} ).rename_vars( 
    {"Pcp": "precip", "XTIME": "time", "XLONG": "longitude",
    "XLAT": "latitude"} )

ds_2.to_netcdf("../temp/CHIRPS_interp_WRF.nc" )

In [15]:
import xarray as xr
import xesmf as xe

# Cargamos el archivo de WRF.
ds_1 = xr.open_dataset("../temp/prec_mroc_hist_hist_days.nc")
ds_1["XLONG"] = ds_1["XLONG"].isel(south_north = 0)
ds_1["XLAT"] = ds_1["XLAT"].isel(west_east = 0)
ds_1 = ds_1.swap_dims( {"west_east": "XLONG", "south_north": "XLAT"}
    ).drop_vars("XTIME_bnds")

# Quitamos celdas de la orilla.
k = 1
ds_1 = ds_1.isel( { "XLAT": slice(k, -k), "XLONG": slice(k, -k) } )

# Cargamos el archivo de CHIRPS.
ds_2 = xr.open_dataset( "../temp/CHIRPS_comp_hist.nc")
ds_2 = ds_2.drop_vars( ["precip", "time"] ).rename(
    {"longitude": "XLONG", "latitude": "XLAT"} )

# Cambiamos la resolución.
regridder = xe.Regridder(ds_1, ds_2, "conservative")
ds_out = regridder(ds_1, keep_attrs=True)

# Guardamos el archivo.
ds_out.to_netcdf("../temp/WRF_regrid_CHIRPS_days.nc" )

ds_out

<xarray.Dataset>
Dimensions:  (XTIME: 10957, XLAT: 52, XLONG: 64)
Coordinates:
  * XTIME    (XTIME) datetime64[ns] 1985-01-01T11:30:00 ... 2014-12-31T09:00:00
  * XLONG    (XLONG) float32 -100.5 -100.4 -100.4 ... -97.43 -97.38 -97.33
  * XLAT     (XLAT) float32 18.12 18.17 18.22 18.27 ... 20.52 20.57 20.62 20.67
Data variables:
    Pcp      (XTIME, XLAT, XLONG) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
Attributes: (12/143)
    CDI:                             Climate Data Interface version 2.1.0 (ht...
    Conventions:                     CF-1.6
    TITLE:                            OUTPUT FROM WRF V4.3 MODEL
    START_DATE:                      1984-09-27_00:00:00
    SIMULATION_START_DATE:           1984-01-01_00:00:00
    WEST-EAST_GRID_DIMENSION:        133
    ...                              ...
    HYBRID_OPT:                      2
    ETAC:                            0.2
    frequency:                       day
    history:                         Wed Jan 31 16:06:27 2024: cdo daysum pre...
    CDO:                             Climate Data Operators version 2.1.0 (ht...
    regrid_method:                   conservative